In [ ]:
from __future__ import print_function
import h5py
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input, Concatenate, Reshape
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.utils import plot_model
from keras.models import Model
from keras.optimizers import adadelta as ada
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import copy
import pydot
from keras.utils import multi_gpu_model
from sklearn.model_selection import StratifiedKFold


%matplotlib inline

In [ ]:
hdf5_path = 'Data/dataset.hdf5'
subtract_mean = True
hdf5_file = h5py.File(hdf5_path, "r")
if subtract_mean:
    mm = hdf5_file["train_mean"][0, ...]
    mm = mm[np.newaxis, ...]
data_num = hdf5_file["train_flow"].shape[0]
    

    
num_classes = 2
epochs = 30

flow_rows, flow_cols = 298, 17

    

x_train = hdf5_file["train_flow"][:,...]
if subtract_mean:
    x_train -= mm

y_train = hdf5_file["train_labels"][:, ...]

hdf5_file.close()
    
hdf5_path = 'Data/dataset-IoT.hdf5'
hdf5_file = h5py.File(hdf5_path, "r")

x_test = hdf5_file["IoT_flow"][:,...]
if subtract_mean:
    x_test -= mm

y_test = hdf5_file["labels"][:, ...]

hdf5_file.close()





class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)
d_class_weights = dict(enumerate(class_weights))

input_shape = (x_train.shape[1], x_train.shape[2], x_train.shape[3])


y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
batch_size = 1024
epochs = 30
filters= 32
kernel_size= (3,3)
activations= "sigmoid"
pool_size= (2,2)
dropout = 0.29104403739531765
lr = 0.001433773291970261
rmsprop = keras.optimizers.RMSprop(lr=lr)
optim = rmsprop
layers = 2
seed = 42
np.random.seed(seed)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []
for train, test in kfold.split(x_train, y_train):
    model = Sequential()

    model.add(Conv2D(filters, kernel_size=kernel_size,activation=activations, input_shape=input_shape,padding = "same"))
    for i in range(layers-1):
        model.add(Conv2D(filters,kernel_size=kernel_size, activation=activations, padding = "valid"))
        
    model.add(MaxPooling2D(pool_size=(pool_size)))
    model.add(Flatten())
    model.add(Dropout(dropout))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(num_classes, activation='softmax'))
    model.summary()
    try:
        model = multi_gpu_model(model, gpus = 4)
    except:
        pass
    model.compile(loss='binary_crossentropy', optimizer=optim, metrics=['accuracy'])
    model.fit(x_train[train],y_train[train], batch_size=batch_size, epochs=epochs, verbose=0, class_weight=class_weights, shuffle=True)
    scores = model.evaluate(x_train[test], y_train[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
